# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    #print([(nx.average_shortest_path_length(g),nx.average_clustering(g)) for g in P1_Graphs])

    #print([[list(g.degree().values()).count(i) / float(nx.number_of_nodes(
    #g)) for i in sorted(set(g.degree().values()))] for g in P1_Graphs])
    
    # or we can use 
    #print([nx.info(g) for g in P1_Graphs])
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [4]:
#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

def salary_predictions():
    
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(nx.degree(G))
    train = df[(df['ManagementSalary'].isnull() == False)]
    test = df[(df['ManagementSalary'].isnull())].drop(['ManagementSalary'],axis = 1)
    train_y = train['ManagementSalary'] 
    train_X = train.drop(['ManagementSalary'], axis = 1)
    clf = RandomForestClassifier(n_estimators= 50, max_depth=3).fit(train_X, train_y)

    return pd.Series([i[1] for i in clf.predict_proba(test)], index = test.index )

In [21]:
#salary_predictions()

1       0.119554
2       0.572017
5       0.875686
8       0.166246
14      0.148505
18      0.101808
27      0.130907
30      0.205023
31      0.207737
34      0.128261
37      0.166246
40      0.133621
45      0.114858
54      0.305368
55      0.256817
60      0.202224
62      0.855185
65      0.429006
77      0.086149
79      0.103278
97      0.100071
101     0.095994
103     0.202224
108     0.117306
113     0.387894
122     0.064401
141     0.204833
142     0.802580
144     0.141369
145     0.299361
          ...   
913     0.107992
914     0.104812
915     0.029224
918     0.104800
923     0.104136
926     0.105315
931     0.101079
934     0.143850
939     0.083025
944     0.029728
945     0.104136
947     0.102215
950     0.207069
951     0.103492
953     0.088817
959     0.022738
962     0.028165
963     0.170379
968     0.108249
969     0.108249
974     0.097230
984     0.143850
987     0.176463
989     0.104136
991     0.192314
992     0.022738
994     0.033780
996     0.0292

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [22]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [80]:
def new_connections_predictions():
    
    edges = pd.Series(future_connections.index, index = future_connections.index)
    future_connections['common neighbors'] = edges.apply(lambda x : len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['jaccard coefficient'] = edges.apply(lambda x : list(nx.jaccard_coefficient(G, [(x[0], x[1])]))[0][2])
    future_connections['resource allocation'] = edges.apply(lambda x : list(nx.resource_allocation_index(G, [(x[0], x[1])]))[0][2])
    future_connections['adamic adar'] = edges.apply(lambda x : list(nx.adamic_adar_index(G, [(x[0], x[1])]))[0][2])
    future_connections['preferential attch'] = edges.apply(lambda x : list(nx.preferential_attachment(G, [(x[0], x[1])]))[0][2])
    train = future_connections[(future_connections['Future Connection'].isnull() == False)]
    test = future_connections[(future_connections['Future Connection'].isnull())].drop(['Future Connection'],axis = 1)
    train_y = train['Future Connection'] 
    train_X = train.drop(['Future Connection'], axis = 1)
    clf = RandomForestClassifier(n_estimators= 50, max_depth=3).fit(train_X, train_y)
    
    return pd.Series([i[1] for i in clf.predict_proba(test)], index = test.index )

In [81]:
#new_connections_predictions()

(107, 348)    0.021890
(542, 751)    0.016968
(20, 426)     0.528038
(50, 989)     0.016968
(942, 986)    0.016968
(324, 857)    0.016968
(13, 710)     0.210162
(19, 271)     0.151861
(319, 878)    0.016968
(659, 707)    0.016968
(49, 843)     0.016968
(208, 893)    0.016968
(377, 469)    0.016968
(405, 999)    0.016968
(129, 740)    0.016968
(292, 618)    0.061713
(239, 689)    0.016968
(359, 373)    0.016968
(53, 523)     0.020854
(276, 984)    0.016968
(202, 997)    0.016968
(604, 619)    0.097193
(270, 911)    0.016968
(261, 481)    0.069225
(200, 450)    0.876124
(213, 634)    0.016968
(644, 735)    0.085368
(346, 553)    0.016968
(521, 738)    0.016968
(422, 953)    0.016968
                ...   
(672, 848)    0.016968
(28, 127)     0.949180
(202, 661)    0.016968
(54, 195)     0.973013
(295, 864)    0.016968
(814, 936)    0.016968
(839, 874)    0.016968
(139, 843)    0.016968
(461, 544)    0.016968
(68, 487)     0.016968
(622, 932)    0.016968
(504, 936)    0.016968
(479, 528) 